# Reinforcement Learning Application in UNO Game

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import torch
sys.path.append("./")

## Introduction

For our project, we want to apply Reinforcement Learning techniques to the game of Uno. We will consider 1v1 UNO with two key assumptions: first we have an infinite number of decks, and we must finish the current deck before moving onto the next. Second, the game will always terminate, meaning there are no ties).

### State ($\mathcal{S}$)
One naive representation for the Uno is utilizing tabular methods, which stores all possibiles states in a table. However, if we wanted to accurately model the game of Uno, the state space is too large to be able to efficiently run tabular methods. In our initial research, we determined that if do not account for the value of the card (and not the color), we would have approximately $~130M$ possible states. Therefore, we will reduce the state space through the following simplification:

- Note: In a deck of Uno cards, there are 60 unique cards with 2 of each type.
- $4$ planes where each entry of a plane corresponds to a given card and 3 of those planes to having a given number of a specific card (you can either have 0, 1, or 2 of each card); the last plane is the target / open card.
- Each plane is of dimension $(4, 15)$, representing $4$ colors and $15$ possible card values. 
- The encoding is binary for all planes.

- Motivations of using `function approximation`: state space is way too big.

### Action ($\mathcal{A}$)
In total, there are $61$ different actions a player can take, to which $4 \times 15 = 60$ of them are for playing different cards and $1$ action for drawing a card from the deck. However, only a few of them are legal actions depending on different states, which will be checked carefully in our game simulation and training.

### Reward ($\mathcal{R}$) 
The reward is defined as $+1$ for winning, $−1$ for losing, and $0$ for all intermediate states. Based on our assumptions, the game must terminate, so all final rewards are either $+1$ or $-1$.

## Model Architecture

In order to better model the state space, we chose a standard _two-layer fully connected_ NN as our approximation model. Specifically, we have tried out different combinations of the number of layers and hidden dimension, but we found that a hidden dimension of $512$ yields the best performance. Note in the diagram below that we have an input of $4 * 4* 15 = 240$ parameters for the state representation, and the output has $61$ parameters—one for each action.
<!-- 
- `nn.Linear(4 x 4 x 15, 512)`
- `nn.ReLU()`
- `nn.Linear(512, 512)`
- `nn.ReLU()`
- `nn.Linear(512, 61)` -->
<center>
<img style="height: auto; width: 60%;" class="img" src="log/neural-network.png" >
</center>

In [23]:
from model.backbone import Q
from torchinfo import summary
model = Q()
summary(model.model, (1, 4 * 4 * 15))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 61]                   --
├─Linear: 1-1                            [1, 512]                  123,392
├─ReLU: 1-2                              [1, 512]                  --
├─Linear: 1-3                            [1, 512]                  262,656
├─ReLU: 1-4                              [1, 512]                  --
├─Linear: 1-5                            [1, 61]                   31,293
Total params: 417,341
Trainable params: 417,341
Non-trainable params: 0
Total mult-adds (M): 0.42
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 1.67
Estimated Total Size (MB): 1.68

## Baseline Agent

`Random Agent` acts by randomly choosing one action from available legal actions.

By playing two `Random Agents` against each other, we observe that the advantage given by playing first is not negligible: the first player that has winning rate around $51\%$. This is not surprising though: in a game where the goal is to let go of your cards as fast as possible, playing first will allow you to getting rid of cards first, and this will result in higher reward in the long run.

While we do note that the increase of the winning rate is small, we also understand that UNO is a highly stochastic game, and thus the advantage of being the first player is not as significant as what we have seen in Tic-Tac-Toe. There is only a slight advantage of $2\%$ in terms of winning rate. With this observation, we have choosen to train all of our algorithms as the second player as this is a relatively more difficult task and may result in better performance.

In [16]:
import numpy as np
np.random.seed(2023)
from tests.eval import test_trained_agents
from uno.agents.random_agent import RandomAgent

random_agent = RandomAgent(61)
rewards = test_trained_agents(random_agent, random_agent, 10000, True)

100%|██████████| 10000/10000 [01:00<00:00, 165.56it/s]


------------------------------------------------------------
Average Rewards
------------------------------------------------------------
RANDOM Agent Average Reward: 0.0192
RANDOM Agent Average Reward: -0.0192

------------------------------------------------------------
Total Number of Games: 10000
RANDOM Agent wins 5096 games (RANDOM Agent win rate: 50.96%)
RANDOM Agent wins 4904 games (RANDOM Agent win rate: 49.04%)
Draws 0 games (Draw rate: 0.0%)


## Hyperparameter Selection

One key aspect of our model's performance is the initialization of the hyperparameters. In order to find the model that results in the best trained agent, we experimented with $3 * 2 * (2 + 2) = 24$ sets of parameters for each algorithm for $50000$ episodes. For the case where we set $\epsilon=0.95$, we exponentially decayed it every $100$ episodes by a factor of $\kappa$.

<center>

| Hyperparameters   |      Short Description     |  Candidate Values List |
|:-----------------:|:-------------------:|:----------------------:|
| $\eta$ |  Learning Rate | $[10^{-2}, 10^{-3}, 10^{-4}]$ |
| $\gamma$ |  Discount Factor | $[0.95, 0.99]$ |
| $\epsilon$ |    Exploration vs. Exploitation  |   $[0.05, 0.10,0.95]$ |
| $\kappa$ (optional) | Control Decay of $\epsilon$ (required when $\epsilon = 0.95$) |    $[0.90, 0.95]$ |

</center>

To clarify, the parameters such as the number of training episodes and frequency of decaying $\epsilon$ can also be regarded as hyperparameters. However, through numerous experiments, we finally decided to train every model for $200K$ episodes, and if we are using $\epsilon=0.95$, it should be decayed every $2000$ episodes.

**Note.** Unfortunately, due to high randomness, training $200K$ episodes of UNO takes more than 8 hours on our laptop, so we are not able to train an agent for longer due to limited resources.


### Evaluation
Every $1000$ episodes during training, we evaluate our training by simulating $10K$ games between the `Random Agent` and our learned model. This allows us to see if there is an upward trend in the average reward and winning rate. In addition, we base the hyperparameter selection on this metric.

## Agents & Algorithms

### Algorithm I: Monte Carlo On Policy Approximation

We trained our `MC Agent` for $200K$ episodes with $\eta=10^{-4}$, $\gamma=0.95$, $\epsilon=0.95$, and decay rate $\kappa=0.95$. In general, we did not see big performance gain for the MC agent, although there is about $1.5\%  - 2\%$ of improvement. Relative to one of the references we found in our initial research, this is actually on par with their results. We suspect that the lack of a significant improvement may be rooted in our hyperparameter selection.


<div class='container'>
<img style="height: auto; width: 45%;" class="img" src="log/MC/mc-agent-[200000]-[0.0001]-[0.95]-[0.95]-[first].png" />
&nbsp;
&nbsp;
<img style="height: auto; width: 45%;" class="img" src="log/MC/mc-agent-[200000]-[0.0001]-[0.95]-[0.95]-[second].png" /></div>

In [15]:
from tests.tests import *
from uno.agents.mc_agent import MCAgent
mc_agent = CHECKPOINTS['MC Agent']
rewards = test_trained_agents(mc_agent, random_agent, 10000, True)

------------------------------------------------------------
Average Rewards
------------------------------------------------------------
MC Agent Average Reward: 0.0384
RANDOM Agent Average Reward: -0.0384

------------------------------------------------------------
Total Number of Games: 10000
MC Agent wins 5192 games (MC Agent win rate: 51.92%)
RANDOM Agent wins 4808 games (RANDOM Agent win rate: 48.08%)
Draws 0 games (Draw rate: 0.0%)


In [14]:
rewards = test_trained_agents(random_agent, mc_agent, 10000, True)

------------------------------------------------------------
Average Rewards
------------------------------------------------------------
RANDOM Agent Average Reward: -0.0258
MC Agent Average Reward: 0.0258

------------------------------------------------------------
Total Number of Games: 10000
RANDOM Agent wins 4871 games (RANDOM Agent win rate: 48.71%)
MC Agent wins 5129 games (MC Agent win rate: 51.29%)
Draws 0 games (Draw rate: 0.0%)


### Algorithm II: SARSA

Next, we trained our `SARSA Agent` for $200K$ episodes with $\eta=10^{-4}$, $\gamma=0.95$, $\epsilon=0.95$, and decay rate $\kappa=0.95$. Surprisingly, this is our best trained again in terms of the winning rate. In general, the `SARSA Agent` achieved above a $56\%$ winning rate in $10000$ simluations. Relative to the stochasticity of Uno, this rate is considered to be a drastic increase in playing performance.

Furthermore, in one of our references, they foudn that their winning rate of their `SARSA Agent` is roughly $52\%$, insinuating that there is validity and a certain amount of accuracy in our state representation and training strategy.


<div class='container'>
<img style="height: auto; width: 45%;" class="img" src="log/SARSA/sarsa-agent-[200000]-[0.0001]-[0.95]-[0.95]-[first]-[0].png" />
&nbsp;
&nbsp;
<img style="height: auto; width: 45%;" class="img" src="log/SARSA/sarsa-agent-[200000]-[0.0001]-[0.95]-[0.95]-[second]-[0].png" /></div>
</div>

In [4]:
from uno.agents.sarsa_agent import SARSAAgent
sarsa_agent = CHECKPOINTS['SARSA Agent']
rewards = test_trained_agents(random_agent, sarsa_agent, 10000, True)

100%|██████████| 10000/10000 [01:42<00:00, 97.60it/s]


------------------------------------------------------------
Average Rewards
------------------------------------------------------------
RANDOM Agent Average Reward: -0.1204
SARSA Agent Average Reward: 0.1204

------------------------------------------------------------
Total Number of Games: 10000
RANDOM Agent wins 4398 games (RANDOM Agent win rate: 43.98%)
SARSA Agent wins 5602 games (SARSA Agent win rate: 56.02%)
Draws 0 games (Draw rate: 0.0%)


In [5]:
rewards = test_trained_agents(sarsa_agent, random_agent, 10000, True)

100%|██████████| 10000/10000 [01:26<00:00, 115.85it/s]


------------------------------------------------------------
Average Rewards
------------------------------------------------------------
SARSA Agent Average Reward: 0.141
RANDOM Agent Average Reward: -0.141

------------------------------------------------------------
Total Number of Games: 10000
SARSA Agent wins 5705 games (SARSA Agent win rate: 57.05%)
RANDOM Agent wins 4295 games (RANDOM Agent win rate: 42.95%)
Draws 0 games (Draw rate: 0.0%)


### Algorithm III: REINFORCE

We trained our `REINFORCE Agent` for $70K$ episodes with $\gamma=0.95$ and $\eta=10^{-4}$. Since the REINFORCE algorithm is based on a different methodology and we found that a `REINFORCE Agent` able to win only about $51\%$ of the time, we suspect that REINFORCE may not be the best RL algorithm for Uno.

<!-- However, the results are not promising. In this algorithm, we forgot to use $\epsilon$-greedy so this might explain why `REINFORCE Agent` is not working as desired. We are training a new one with $\epsilon$-greedy strategy now and hopefully it will give us better results. -->
<!-- Average Rewards

------------------------------------------------------------
Reinforce Agent Average Reward: 0.0038
RANDOM Agent Average Reward: -0.0038

------------------------------------------------------------
Total Number of Games: 10000
Reinforce Agent wins 5019 games (Reinforce Agent win rate: 50.19%)
RANDOM Agent wins 4981 games (RANDOM Agent win rate: 49.81%)
Draws 0 games (Draw rate: 0.0%) -->

<!-- Very inconclusive -->
<!-- ![avg_reward_reinforce.png](checkpoint/REINFORCE/avg_reward_reinforce.png) -->

<div class='container'>
<img style="height: auto; width: 45%;" class="img" src="log/REINFORCE/reinforce_plays_first.png" />
&nbsp;
&nbsp;
<img style="height: auto; width: 45%;" class="img" src="log/REINFORCE/reinforce_plays_second.png" /></div>
</div>

In [12]:
from tests.tests import *
from uno.agents.reinforce_agent import ReinforceAgent
reinforce_agent = CHECKPOINTS['REINFORCE Agent']
rewards = test_trained_agents(random_agent, reinforce_agent, 10000, True)

------------------------------------------------------------
Average Rewards
------------------------------------------------------------
RANDOM Agent Average Reward: -0.0212
REINFORCE Agent Average Reward: 0.0212

------------------------------------------------------------
Total Number of Games: 10000
RANDOM Agent wins 4894 games (RANDOM Agent win rate: 48.94%)
REINFORCE Agent wins 5106 games (REINFORCE Agent win rate: 51.06%)
Draws 0 games (Draw rate: 0.0%)


In [13]:
rewards = test_trained_agents(reinforce_agent, random_agent, 10000, True)

------------------------------------------------------------
Average Rewards
------------------------------------------------------------
REINFORCE Agent Average Reward: 0.0132
RANDOM Agent Average Reward: -0.0132

------------------------------------------------------------
Total Number of Games: 10000
REINFORCE Agent wins 5066 games (REINFORCE Agent win rate: 50.66%)
RANDOM Agent wins 4934 games (RANDOM Agent win rate: 49.34%)
Draws 0 games (Draw rate: 0.0%)


### Algorithm IV: Q-Learning

#### Double Deep Q_Network

The Double Deep Q-Network (Double DQN) is an extension of the standard Deep Q-Network (DQN) that aims to reduce the overestimation bias often found in Q-learning algorithms. In traditional DQN, the same network is used to both select and evaluate the best policy, leading to an overoptimistic estimation of action values. To mitigate this issue, the Double DQN exploits two separate networks: one for action selection and another for action evaluation. During each iteration, the agent plays n games using ε-greedy exploration and generates a set of trajectories (S, A, R, S'). The Double DQN update rule then utilizes both networks by selecting the action with the highest Q-value from the first network and evaluating that action using the second network. This decouples the action selection and evaluation processes, thus reducing the overestimation bias and improving the stability of learning.

The `DQN Agent` had the following hyperparameters: $\eta=10^{-4}$, $\gamma=0.95$, $\epsilon=0.95$. Overall, we notice a slight increase to its winning rate at $51\%$, but nothing on the scale of the `SARSA Agent.` DQN agent was trained with a set of different parameters:
Architecture: one hidden layer with size $128$, we are able to achieve 200K training in a reasonable time.



<div class='container'>
<img style="height: auto; width: 45%;" class="img" src="log/DQN/dqn-agent-[200000]-[0.0001]-[no decay_0.08]-[0.95]-[first].png" />
&nbsp;
&nbsp;
<img style="height: auto; width: 45%;" class="img" src="log/DQN/dqn-agent-[200000]-[0.0001]-[no decay_0.08]-[0.95]-[second].png" /></div>
</div>

In [16]:
from tests.tests import *
from uno.agents.dqn_agent import DQNAgent
dqn_agent = CHECKPOINTS['DQN Agent']
rewards = test_trained_agents(dqn_agent, random_agent, 10000, True)

------------------------------------------------------------
Average Rewards
------------------------------------------------------------
DQN Agent Average Reward: 0.029
RANDOM Agent Average Reward: -0.029

------------------------------------------------------------
Total Number of Games: 10000
DQN Agent wins 5145 games (DQN Agent win rate: 51.45%)
RANDOM Agent wins 4855 games (RANDOM Agent win rate: 48.55%)
Draws 0 games (Draw rate: 0.0%)


In [17]:
rewards = test_trained_agents(random_agent, dqn_agent, 10000, True)

------------------------------------------------------------
Average Rewards
------------------------------------------------------------
RANDOM Agent Average Reward: -0.0148
DQN Agent Average Reward: 0.0148

------------------------------------------------------------
Total Number of Games: 10000
RANDOM Agent wins 4926 games (RANDOM Agent win rate: 49.26%)
DQN Agent wins 5074 games (DQN Agent win rate: 50.74%)
Draws 0 games (Draw rate: 0.0%)


However the results is not as expected.
So we expanded our architecture to two hidden layer with size 128 and 512
With a 70K episode training, we are able to get the below graph.(new graph) 

With a slightly better average reward and average wining rate, and mainly above the 50% line

<div class='container'>
<img style="height: auto; width: 45%;" class="img" src="log/DQN/1.jpeg" />
&nbsp;
&nbsp;
<img style="height: auto; width: 45%;" class="img" src="log/DQN/2.jpeg" /></div>
</div>

## Contests

After training each agent, we played them against each other. Due to the overwhelming increase of winning rate for the `SARSA Agent`, we see that it wins the contest, both as the first player and as the second player. For clarity, the percentages listed in the table below represent the winning rate of the row player, given that the row player goes first.

In [14]:
import numpy as np
np.random.seed(2023)
from tests.tests import contests
from tests.eval import *
stats = contests(n=1000)
stats

100%|██████████| 1000/1000 [00:14<00:00, 70.95it/s]


,Random Agent,SARSA Agent,MC Agent,REINFORCE Agent,DQN Agent
Random Agent,50.70%,45.30%,48.60%,51.30%,51.00%
SARSA Agent,57.70%,52.40%,56.30%,56.00%,53.80%
MC Agent,50.20%,45.80%,48.20%,53.00%,50.70%
REINFORCE Agent,48.90%,42.30%,48.80%,51.10%,47.70%
DQN Agent,52.20%,43.20%,52.10%,51.20%,47.90%


## Discussion & Limitation

### More on Hyperparameters & Architecture

A majority of the agents we trained do not seem to have any edge over the random agent, which may seem rather counterintuitive, especially when we realize that UNO is not complicated game. A lot of parameters affect the performance of the agents we propose here, namely the architecture of our NNs and the hyperparameters.

Most of our fine-tuning is heavily inspired by this paper (Winning UNO with Reinforcement Learning). In this regard, we believe a two-layer connected neural network should be deep enough to report good results for all of the algorithms we used. It is worth noting that the hyperparameters they use give a lot of weight to initial exploration throughout the training even with their decay factor. For example, setting $\epsilon =  0.95$ and $\kappa = 0.995$ with decay occurring 10th of the way still results in a very explorative behavior— $\epsilon \times \kappa^{10} \approx 0.90$). We deem the discount rate is to be ratherhigh ($\gamma = 0.95$), considering winning rapidly or in a long time does not matter that much in the end, and the learning rate is $\alpha=0.0001$.

The limited successes of some of our agents might stem from this aversion of some of the algorithms to exploration. For instance, REINFORCE does not give way to exploration. As the state space is really large, the policy might focus too much on specific episodes and states which have repeated throughout the training.

### Multiple players

We have pondered over adding another random agent to solidify the training of our agents. Although the drawbacks of such a solution seem obvious (training will take inevitably longer - why should it be any different from 1v1 in terms of policy), adding players might result in more variance in the states covered throughout the episodes as there is more interaction between all players. In the end, this could reflect a better exploration throughout the training and eventually report better performance results.

#### Two more base agents

Some basic strategies come to mind for other baseline agents: one, which is widely played, is to play a card of the same value whenever it is possible, regardless the color is (denoted as the "value"-strategy), and an other one could be to play the target color whenever possible (denoted as the "color"-strategy). The value-strategy usually pans out better in the end because there is there are more cards to play based on color (roughly 1/4 of the deck) versus to play based on value (roughly 1/15).

We havent added the "saying UNO" part but the obvious modeling is adding a Bernoulli variable when left with one card only: as this would be symmetric for all players, it wouldn't change the average reward, and would just take longer training cause of the variance added.

### State representation

There are a lot of possible state representations of UNO. Choosing a good representation lies in reducing the complexity of the training as much as possible while still reflecting the dynamics of the game. We have thought about adding randomness to colors and values (as in, for example, every standard card could have a 1/4 chance to be of a given color every time) to get rid of these dimensions, and then building this agent over a baseline agent that understands these colors and values matchings. But it seemed too distant from the actual game, and maybe not reflect how one would go about playing Uno.

### Game is difficult

Despite these modifications, Uno is a very stochastic game, and agents struggle to win over 60% of their games against random agents. If we had unlimited resources in terms of CPU and time, we might be able to achieve better results by letting each agent train for more than 200K games. Nonetheless, we do see that there are still nonneglibible improvements by using the RL techniques we've learned in class.

